In [3]:
import numpy as np
import json

# Setting Up Functions

In [4]:
def downscale(data, factor):
  downscaled = data.reshape(data.shape[0] // factor, factor,
                            data.shape[1] // factor, factor,
                            data.shape[2] // factor, factor).mean(axis=5)
  downscaled = downscaled.mean(axis=3)
  downscaled = downscaled.mean(axis=1)
  return downscaled

In [5]:
def downscale_by_skipping(data, factor):
  new_size = (data.shape[0] // factor) * (data.shape[1] // factor) * (data.shape[2] // factor)
  downscaled = data.flatten()[::pow(factor, 3)][:new_size].reshape(data.shape[0] // factor,
                                                                   data.shape[1] // factor,
                                                                   data.shape[2] // factor)
  return downscaled

# Head Scan

## Normal

In [ ]:
scans = []
for i in range(112, 0, -1):
  scan = np.fromfile(f"./skull/CThead{i}.txt", dtype=">u2")
  scan = scan.reshape(256, 256)
  scans.append(scan)

full_data = np.array(scans, dtype=">u2")

In [ ]:
downscaled = downscale(full_data, 4)
downscaled = (downscaled - downscaled.min()) / (downscaled.max() - downscaled.min())
l = list(downscaled.flatten())
json_output = json.dumps({
    "dimensions": list(downscaled.shape),
    "data":l
})
output = open("./skullData.json", "w")
output.write(json_output)
output.close()

## Z-Scores

In [ ]:
flattened = full_data.flatten()
original_shape = full_data.shape
non_zero_mean = flattened[flattened > 0].mean()
non_zero_std = flattened[flattened > 0].std()
full_data_z_scores = (full_data - non_zero_mean) / non_zero_std
zero_z_score = -non_zero_mean / non_zero_std

flattened_z_scores = full_data_z_scores.flatten()
flattened_z_scores[flattened_z_scores == zero_z_score] = 0
full_data_z_scores = flattened_z_scores.reshape(original_shape)
full_data_z_scores.shape

(112, 256, 256)

In [ ]:
downscaled = downscale(full_data_z_scores, 4)
downscaled = (downscaled - downscaled.min()) / (downscaled.max() - downscaled.min())
downscaled = downscaled ** 2
l = list(downscaled.flatten())
json_output = json.dumps({
    "dimensions": list(downscaled.shape),
    "data":l
})
output = open("./skullDataZScores.json", "w")
output.write(json_output)
output.close()

# Terracotta Bunny

## Normal

In [9]:
scans = []
for i in range(360, 0, -1):
  scan = np.fromfile(f"./bunny_data/{i}", dtype=">u2")
  scan = scan.reshape(512, 512)
  scans.append(scan)

full_data = np.array(scans, dtype=">u2")

In [10]:
full_data[full_data == full_data.max()] = 0

In [11]:
downscaled = downscale(full_data, 8)
downscaled = (downscaled - downscaled.min()) / (downscaled.max() - downscaled.min())
l = list(downscaled.flatten())
json_output = json.dumps({
    "dimensions": list(downscaled.shape),
    "data":l
})
output = open("./bunnyData.json", "w")
output.write(json_output)
output.close()